In [12]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import pandas as pd
import math
import numpy as np
import os
import time 

import cntk as C
import cntk.tests.test_utils
from cntk.layers import *
from cntk.layers.typing import *
import pickle
import random
from cntk import sequence
from cntk import load_model
from cntk.device import try_set_default_device, gpu,cpu
from scipy.sparse import csr_matrix

from gensim.models import Word2Vec
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components
try_set_default_device(gpu(0))


vocab_size = 80000
num_labels = 19#19
title_size = 52000
body_size  = 210000
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200

max_length_title = 53
max_length_body  = 200

suffix = "180days_all_shuffled"
#suffix = "linkedin_only"
prefix = "/home/t-haohu/IndustryClassifier/Data/"

#data_token_body        = "{}/middle/{}_token_body.txt".format(prefix,suffix)
data_train_sample = "{}/middle/train_{}.txt".format(prefix,suffix)
#data_train_sample = "{}/middle/train_{}_with_linkedin_all.txt".format(prefix,suffix)
data_test_sample  = "{}/middle/test_{}.txt".format(prefix,suffix)
#data_test_sample_editor  = "{}/middle/test_{}_editor.txt".format(prefix,suffix)

data_title_sample    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_sample     = "{}/ready/body_{}.wl".format(prefix,suffix)
suffix = "180days_all_shuffled"
data_industry_sample = "{}/ready/industry_{}.wl".format(prefix,suffix)
filter_num=200 
dropout_rate = 0.5
emb_dim =300

def load_data(input_file,title_dict,industry_dict):
    data = open(input_file, encoding = "utf-8").readlines()
    
    data_title =[ [] for x in range(len(data))]#np.zeros((len(data),max_length_title),dtype = np.float32)
    data_label = np.zeros((len(data),1),dtype = np.float32)
    
    for index,line in enumerate(data):
        row = line.strip("\n").split("\t")       
        title    =  row[0]
        industry =  row[1]
        
        for jndex,token in enumerate(title.split(" ")):
            if jndex>=max_length_title:
                break
            data_title[index].append(title_dict.get(token,len(title_dict)-1))
        while len(data_title[index])<3:
            data_title[index].append(len(title_dict)-1)
        data_label[index] = industry_dict.get(industry,len(industry_dict))
    data_title = [ np.array(x) for x in data_title]
    return data_title,data_label

def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(sequence.last(F(x)), sequence.first(G(x)),name='h2')
    return apply_x


def load_embedding(title_file,embedding_model_file):
    model = Word2Vec.load(embedding_model_file)
    title_list = [x.strip("\n") for x in open(title_file,encoding = 'utf-8').readlines()]
    embedding = np.zeros((len(title_list),emb_dim))
    count = 0
    for i,w in enumerate(title_list):
        try:
            vec = model.wv[w]
        except:
            vec=model.wv["UNK"]
            count+=1
        embedding[i] =vec
    print(count)
    return embedding


def create_model_lstm(embed = False):
    #version 2 : 1 dense layer version3: sigmoid activation in dens
    with C.layers.default_options(initial_state=0.1):
        if embed:
            h1= C.layers.Sequential([
            C.layers.Embedding(emb_size,name='embed_1'),
            #C.to_sequence(),
            C.layers.BatchNormalization(),
            C.layers.Stabilizer()])(input_xt_one_hot)
        else:
            h1= C.layers.Sequential([
            C.layers.Embedding(emb_dim,name='embed_2'),
            C.layers.BatchNormalization(),
            C.layers.Stabilizer()])(input_xt_one_hot)       
        h2=BiRecurrence(C.layers.LSTM(hidden_dim),C.layers.LSTM(hidden_dim))(h1)
        h4=C.layers.Dense(num_labels, name='classify')(h2)
    return h4
def create_model_cnn_2fold():
    #version 2 : 1 dense layer version3: sigmoid activation in dense
    #
    with C.layers.default_options(initial_state=0.1):


        h1_1= C.layers.Embedding(weights=embedding,name='embed_1')(input_xt_one_hot)#
        h1_2= C.layers.Embedding(300,name='embed_2')(input_xt_one_hot)#init=embedding,
        
        h1_1_expand = C.expand_dims(h1_1,-3)
        h1_2_expand = C.expand_dims(h1_2,-3)
        
        h1 = C.splice(h1_1_expand,h1_2_expand,axis = -3)
   #value,valid = to_static(h1)

        filter_num=100

        h2_1=C.layers.Convolution((3,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(h1)
        h2_2=C.layers.Convolution((4,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(h1)
        h2_3=C.layers.Convolution((5,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(h1)

        h3_1=C.layers.MaxPooling((max_length_title-2,1),name='pooling_1')(h2_1)
        h3_2=C.layers.MaxPooling((max_length_title-3,1),name='pooling_2')(h2_2)
        h3_3=C.layers.MaxPooling((max_length_title-4,1),name='pooling_3')(h2_3)

        h3=C.splice(h3_2,h3_1,h3_3,axis=0)
        drop1 =C.layers.Dropout(0.5)(h3)
        h4=C.layers.Dense(num_labels,name='hidden')(drop1)
    return h4


def batch_iter(data,batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    print('data_size: ', data_size, 'batch_size: ', batch_size, 'num_batches_per_epoch: ', num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            random.shuffle(data)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[start_index:end_index]
            

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_sample).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_sample).readlines()])}




input_xt = C.input_variable(**Sequence[Tensor[1]])
#input_xt = C.input_variable(**Sequence[Tensor[1]])
input_y  = C.input_variable(shape=(1)               ,  dtype=np.int)

input_xt_one_hot = C.one_hot(input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
input_y_one_hot = C.one_hot(input_y  , num_classes=len(industry_dict) ,  sparse_output=True)


test_data  = load_data(data_test_sample,title_dict,industry_dict)
train_data = load_data(data_train_sample,title_dict,industry_dict)
#test_data_editor  = load_data(data_test_sample_editor,title_dict,industry_dict)
embedding = load_embedding(data_title_sample,"word2vec.model")
def test(batch_size,model,data):
    scores = model(input_xt)
    predict = C.argmax(scores,axis = 0)
    confuse = np.zeros((num_labels,num_labels))

    test_data_title,test_data_label = data
    batches = batch_iter(list(zip(test_data_title,test_data_label)), batch_size, 1)
    #print(predict)
    for batch in batches:
        batch_data_title,batch_data_label = zip(*batch)
        batch_data_title = list(batch_data_title)
        predict.eval({input_xt: batch_data_title})
        output = np.array(predict.eval({input_xt: batch_data_title}),dtype=np.int)
        gt = np.array(batch_data_label,dtype=np.int)
        confuse+=fast_hist(output,gt,num_labels)
        
    precision=np.diag(confuse)/np.sum(confuse,axis=0)
    recall = np.diag(confuse)/np.sum(confuse,axis=1)
    accuarcy = np.diag(confuse).sum() / confuse.sum()
    aver_precision=np.nanmean(precision)
    aver_recall = np.nanmean(recall)
   
    print("Precision:{} Recall:{} Acc:{}".format(aver_precision,aver_recall,accuarcy))
    return accuarcy



def train(train_data,num_epochs,learning_rate,batch_size,tag="CNN",l2_weight=0):
    global model
    #learning_rate *= batch_size
    model = create_model_lstm()
    print(C.logging.get_node_outputs(model))
    scores = model(input_xt)

    loss =C.reduce_mean(C.losses.cross_entropy_with_softmax(scores, input_y_one_hot))
    
    # Training
    lr_schedule = C.learning_parameter_schedule(learning_rate)
    #learner = C.adam(scores.parameters, lr=lr_schedule, momentum=0.9,l2_regularization_weight=0)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    momentums = C.momentum_schedule(0.99, minibatch_size=batch_size)
    learner = C.adam(parameters=scores.parameters,#model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15,
                     gradient_clipping_with_truncation=True,
                     l2_regularization_weight=l2_weight)
    trainer = C.Trainer(scores, (loss), [learner], progress_printer)
    
    train_data_title,train_data_label = train_data
    batches = batch_iter(list(zip(train_data_title,train_data_label)), batch_size, num_epochs)

    # training loop
    count = 0
    t = time.time()
    for batch in batches:
        count += 1
        batch_data_title,batch_data_label = zip(*batch)
        batch_data_title = list(batch_data_title)
        #print(type(batch_data_title),type(batch_data_title[0]),batch_data_title[0])
        trainer.train_minibatch({input_xt: batch_data_title, input_y: np.array(batch_data_label)})
        if count%1000== 0:
            print(count,time.time()-t)
            t=time.time()
            acc1=test(batch_size,model,test_data)
            #acc2=test(batch_size,model,test_data_editor)
            
            # save model
            #model.save('./model/{}/{}_acc{:.3f}.dnn'.format(suffix,tag,acc1))
            #model.save('./model/{}/{}_acc1{:.3f}_acc2{:.3f}.dnn'.format(suffix,tag,acc1,acc2))
    

    # save model
    


0


In [ ]:
train(train_data,num_epochs=20,learning_rate=[5e-4*150]*2+[1e-4*150],batch_size = 150,tag = "DeepCNN")



[Output('classify', [#], [19]), Output('h2', [#], [1 x 400]), Output('Block24664_Output_0', [#], [1 x 200]), Output('Block24621_Output_0', [#, *], [1 x 200]), Output('Block24621_Output_1', [#, *], [1 x 200]), Output('PastValue24556_Output_0', [#, *], [1 x 200]), Output('PastValue24559_Output_0', [#, *], [1 x 200]), Output('Block23239_Output_0', [#, *], [1 x 300]), Output('Block23144_Output_0', [#, *], [1 x 300]), Output('embed_2', [#, *], [1 x 300]), Output('OneHotOp22788_Output_0', [#, *], [1 x 56178]), Output('Block24781_Output_0', [#], [1 x 200]), Output('Block24738_Output_0', [#, *], [1 x 200]), Output('Block24738_Output_1', [#, *], [1 x 200]), Output('FutureValue24673_Output_0', [#, *], [1 x 200]), Output('FutureValue24676_Output_0', [#, *], [1 x 200])]
data_size:  768505 batch_size:  150 num_batches_per_epoch:  5124


/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/cntk/core.py:350: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  elif np.issubdtype(sample.dtype, int):
/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "int64", but your input variable (uid "Input22786") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


Learning rate per minibatch: 0.007500000000000001
1000 89.32069253921509
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.5129606778464509 Recall:0.6936419213273262 Acc:0.7088079912557688
2000 227.046813249588
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.6787074612262457 Recall:0.7576608736566911 Acc:0.801505950935147
3000 274.2758710384369
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7373127408795058 Recall:0.7847336722622694 Acc:0.8291352926888511
4000 401.0707335472107
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7690799799909508 Recall:0.797825617450477 Acc:0.8427890454214234
5000 383.6275520324707
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7814816726548525 Recall:0.8101315252858666 Acc:0.850264148651931
6000 399.789489030838
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7832607359379066 Recall:0

Exception ignored in: <bound method DeviceDescriptor.<lambda> of GPU[0] Tesla K80>
Traceback (most recent call last):
  File "/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/cntk/cntk_py.py", line 1012, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt
